In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

data = pd.read_csv('movie_bd_v5.csv')
# создадим словарь для ответов
answers = {}
# наши функции
def select_season(date):
    date = pd.to_datetime(date)
    if(date.month in [12,1,2]):
        return 'winter'
    elif(date.month in [3,4,5]):
        return 'spring'
    elif(date.month in [6,7,8]):
        return 'summer'
    else:
        return 'autumn'

def select_month(date):
    date = pd.to_datetime(date)
    return date.month

def words_count(string):
    string = string.split(' ')
    return len(string)

# 1. У какого фильма из списка самый большой бюджет?
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
display(data.loc[data.budget == data.budget.max()].original_title)

#2. Какой из фильмов самый длительный (в минутах)?
answers['2'] = '2. Gods and Generals (tt0279111)'
display(data.loc[data.runtime == data.runtime.max()].original_title)

#3. Какой из фильмов самый короткий (в минутах)?
answers['3'] = '3. Winnie the Pooh (tt1449283)'
display(data.loc[data.runtime == data.runtime.min()].original_title)

#4. Какова средняя длительность фильмов?
answers['4'] = '2. 110'
display(data.runtime.mean())

#5. Каково медианное значение длительности фильмов?
answers['5'] = '1. 107'
display(data.runtime.median())

#6. Какой самый прибыльный фильм?
# определим столбец прибыль(убытки) как прибыль = сборы - бюджет
data['profit'] =  data['revenue'] - data['budget']
answers['6'] = '5. Avatar (tt0499549)'
display(data.loc[data.profit == data.profit.max()].original_title)

#7. Какой фильм самый убыточный?
answers['7'] = '5. The Lone Ranger (tt1210819)'
display(data.loc[data.profit == data.profit.min()].original_title)

#8. У скольких фильмов из датасета объем сборов оказался выше бюджета?
answers['8'] = '1. 1478'
display(len(data.loc[data.profit > 0]))

#9. Какой фильм оказался самым кассовым в 2008 году?
answers['9'] = '4. The Dark Knight (tt0468569)'
year_2008 = data.loc[(data.release_year == 2008)]
display(year_2008.loc[year_2008.profit == year_2008.profit.max()].original_title)

#10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?
answers['10'] = '5. The Lone Ranger (tt1210819)'
years_2012_2014 = data.loc[(data.release_year >= 2012) & (data.release_year <= 2014) ]
display(years_2012_2014.loc[years_2012_2014.profit == years_2012_2014.profit.min()].original_title)

#11. Какого жанра фильмов больше всего?
answers['11'] = '3. Drama'
genres_dict = {}
for film_genres in data['genres']: 
    genres = film_genres.split('|') 
    for genre in genres:
        if (genres_dict.get(genre) == None):
            genres_dict.setdefault(genre)
            genres_dict[genre] = 1
        else:
            genres_dict[genre] += 1
max_genre_val = 0
max_genre = ""
for item in genres_dict.items():
    if item[1] > max_genre_val:
        max_genre_val = item[1]
        max_genre = item[0]
print(max_genre, max_genre_val, sep=' ', end='\n',)

#12. Фильмы какого жанра чаще всего становятся прибыльными?
answers['12'] = '1. Drama'
profit_df = data.loc[data.profit > 0]
genres_dict = {}
for film_genres in profit_df['genres']:
    genres = film_genres.split('|')
    for genre in genres:
        if (genres_dict.get(genre) == None):
            genres_dict.setdefault(genre)
            genres_dict[genre] = 1
        else:
            genres_dict[genre] += 1
max_genre_val = 0
max_genre = ""
for item in genres_dict.items():
    if item[1] > max_genre_val:
        max_genre_val = item[1]
        max_genre = item[0]
print(max_genre, max_genre_val, sep=' ', end='\n',)

#13. У какого режиссера самые большие суммарные кассовые сборы?
answers['13'] = '5. Peter Jackson'
tmp_df = data.copy()
tmp_df.director =  tmp_df.director.str.split('|')
tmp_df = tmp_df.explode('director')
director_df = tmp_df.groupby(by = ['director']).revenue.sum()
display(director_df.loc[director_df == director_df.max()])


#14. Какой режисер снял больше всего фильмов в стиле Action?
answers['14'] = '3. Robert Rodriguez'
tmp_df.genres = tmp_df.genres.str.split('|')
tmp_df = tmp_df.explode('genres')
display(tmp_df.loc[tmp_df.genres == 'Action']['director'].value_counts().index[0])
del tmp_df

#15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?
answers['15'] = '3. Chris Hemsworth'
tmp_df = data.copy()
tmp_df.cast = tmp_df.cast.str.split('|')
tmp_df = tmp_df.explode('cast')
display(tmp_df.loc[tmp_df.release_year == 2012].groupby(by = 'cast').revenue.sum().sort_values(ascending = False).index[0])

#16. Какой актер снялся в большем количестве высокобюджетных фильмов?
answers['16'] = '3. Matt Damon'
tmp_df = tmp_df.loc[tmp_df.budget > tmp_df.budget.mean()]
display(tmp_df.cast.value_counts().index[0])

#17. В фильмах какого жанра больше всего снимался Nicolas Cage?
answers['17'] = '3. Action'
tmp_df.genres = tmp_df.genres.str.split('|')
tmp_df = tmp_df.explode('genres')
NC_films = tmp_df.loc[tmp_df.cast == 'Nicolas Cage']
display(NC_films.genres.value_counts().index[0])
del tmp_df

#18. Самый убыточный фильм от Paramount Pictures
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
tmp_df = data.copy()
tmp_df.production_companies = tmp_df.production_companies.str.split('|')
tmp_df = tmp_df.explode('production_companies')
film = tmp_df.loc[tmp_df.production_companies == 'Paramount Pictures'].sort_values('profit')['original_title'].values[0]
film_id = tmp_df.loc[tmp_df.original_title == film]['imdb_id'].values[0]
print(film, end=' ')
print('(',film_id,')',end='\n', sep='')

#19. Какой год стал самым успешным по суммарным кассовым сборам?
answers['19'] = '5. 2015'
display(data.groupby(by = 'release_year').revenue.sum().sort_values(ascending = False).index[0])

#20. Какой самый прибыльный год для студии Warner Bros?
answers['20'] = '1. 2014'
display(tmp_df.loc[tmp_df.production_companies.str.contains('Warner Bros')].groupby(by = 'release_year')\
        .profit.sum().sort_values(ascending = False).index[0])
del tmp_df

#21. В каком месяце за все годы суммарно вышло больше всего фильмов?
answers['21'] = '4. Сентябрь'
data['month'] = data['release_date'].apply(select_month)
display(data.groupby('month').imdb_id.count().sort_values(ascending = False).index[0])

#22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)
answers['22'] = '2. 450'
data['season'] = data['release_date'].apply(select_season)
display(data.loc[data.season == 'summer'].imdb_id.count())

#23. Для какого режиссера зима – самое продуктивное время года?
answers['23'] = '5. Peter Jackson'
tmp_df = data.copy()
tmp_df.director = tmp_df.director.str.split('|')
tmp_df = tmp_df.explode('director')
display(tmp_df.loc[tmp_df.season == 'winter'].groupby('director').imdb_id.count().sort_values(ascending = False).index[0])
del tmp_df

#24. Какая студия дает самые длинные названия своим фильмам по количеству символов?
answers['24'] = '5. Four By Two Productions'
tmp_df = data.copy()
tmp_df.production_companies = tmp_df.production_companies.str.split('|')
tmp_df = tmp_df.explode('production_companies')
tmp_df['title_len'] = tmp_df.original_title.map(lambda x:len(x))
display(tmp_df.groupby('production_companies').title_len.mean().sort_values(ascending = False).index[0])

#25. Описание фильмов какой студии в среднем самые длинные по количеству слов?
answers['25'] = '3. Midnight Picture Show'
tmp_df['overview_words_count'] = tmp_df['overview'].apply(words_count)
display(tmp_df.groupby('production_companies').overview_words_count.mean().sort_values(ascending = False).index[0])

#26. Какие фильмы входят в 1 процент лучших по рейтингу?
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
display(data.sort_values(by = 'vote_average', ascending = False)[0:int(len(data)/100)].original_title)

#27. Какие актеры чаще всего снимаются в одном фильме вместе?
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
data.cast = data.cast.str.split('|')
# Вариант 1
cast_combin_dict = {}
for cast in data.cast:
    for combination in list(combinations(cast,2)):
        if combination in cast_combin_dict:
            cast_combin_dict[combination] += 1
        else:
            cast_combin_dict.setdefault(combination, 1)
cast_combin_num = cast_combin_dict.get(sorted(cast_combin_dict, key=cast_combin_dict.get)[-1])
print(max(cast_combin_dict, key=cast_combin_dict.get), cast_combin_num, end = '\n')

# Вариант 2
cast_counter = Counter()
for cast in data.cast:
    for combination in list(combinations(cast,2)):
        if combination in cast_counter:
            cast_counter[combination] += 1
        else:
            cast_counter[combination] = 1
display(cast_counter.most_common(1))

print("Ответы:", "\n")
for answer in answers: print(answer, answers[answer], "\n")
print("Всего ответов: ", len(answers), "\n")



723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

1157    Gods and Generals
Name: original_title, dtype: object

768    Winnie the Pooh
Name: original_title, dtype: object

109.6585494970884

107.0

...

'Midnight Picture Show'

599                                   The Dark Knight
118                                      Interstellar
125                                The Imitation Game
9                                          Inside Out
34                                               Room
1183                          The Wolf of Wall Street
128                                         Gone Girl
1191                                 12 Years a Slave
119                           Guardians of the Galaxy
1081    The Lord of the Rings: The Return of the King
1800                                          Memento
370                                         Inception
872                                       The Pianist
138                          The Grand Budapest Hotel
1188                                              Her
28                                          Spotlight
124                                        Big Hero 6
177                            The Fault in Our Stars
Name: original_title, dtype:

('Daniel Radcliffe', 'Rupert Grint') 8


[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

Ответы: 

1 5. Pirates of the Caribbean: On Stranger Tides (tt1298650) 

2 2. Gods and Generals (tt0279111) 

3 3. Winnie the Pooh (tt1449283) 

4 2. 110 

5 1. 107 

6 5. Avatar (tt0499549) 

7 5. The Lone Ranger (tt1210819) 

8 1. 1478 

9 4. The Dark Knight (tt0468569) 

10 5. The Lone Ranger (tt1210819) 

11 3. Drama 

12 1. Drama 

13 5. Peter Jackson 

14 3. Robert Rodriguez 

15 3. Chris Hemsworth 

16 3. Matt Damon 

17 3. Action 

18 1. K-19: The Widowmaker (tt0267626) 

19 5. 2015 

20 1. 2014 

21 4. Сентябрь 

22 2. 450 

23 5. Peter Jackson 

24 5. Four By Two Productions 

25 3. Midnight Picture Show 

26 1. Inside Out, The Dark Knight, 12 Years a Slave 

27 5. Daniel Radcliffe & Rupert Grint 

Всего ответов:  27 

